### SimpleMKL Training

- In scikit-learn we utilized a single kernel (radial basis function) with cross validated bandwidth (via grid search)

- We now want to extend to the case of multiple kernels (linear combination of a basis set)

- Inspiration in our original implementation from here 

     - https://github.com/qintian0321/SimpleMKL_python

In [36]:
import numpy as np
import pandas as pd
from sklearn import svm
from scipy.spatial import distance_matrix

# SimpleMKL optimization objective

### Primal problem 


### Dual Problem

$$\max_\alpha \ \frac{-1}{2} \sum_{i,j} \alpha_i \alpha_j y_i y_j \sum_m d_mK_m(x_i,x_j) +\sum_i \alpha_i$$

s.t. $$\sum_i \alpha_i y_i=0$$ and $$C \geq \alpha_i \geq 0$$



- Coefficient vector $ \alpha$ is importance of observed features on classification problem




### Abstract Kernel Class 

In [37]:
class Kernel():
    """ Abstract method to compute the kernel matrix under gaussian kernel or polynomial kernel"""
    
    def __init__(self,kernel_type,order=None,bandwidth=None):
        self.kernel_type=kernel_type
        self.order=order
        self.bandwidth=bandwidth
    
    def compute_kernel(self,X):
        
        if self.kernel_type=='linear':
            return np.dot(X,X.T)
        
        
        if self.kernel_type=='gaussian':
            # Compute the distance matrix which is then numerically transformed to gaussian kernel
            scale=distance_matrix(X,X,p=2)
            return np.exp(-0.5*self.bandwidth*(scale**2))
        
        if self.kernel_type=='polynomial':
            return np.dot(X,X.T)**self.order
        

### Functions for Multiple Kernel Optimization

In [93]:
def compute_dual(X,y,kernel_list,d_m,constant,compute_gap=True):
    """ Compute dual objective value
    """
    kernel=compose_kernels(X,kernel_list,d_m)
    single_kernel=svm.SVC(C=constant,kernel='precomputed')
    single_kernel.fit(kernel,y)
    
    alpha=np.empty(len(y))
    alpha[single_kernel.support_]=np.abs(single_kernel.dual_coef_[0]) 
    alpha[alpha==None]=0
    
    
    J=0.5*np.dot(np.dot(alpha,kernel*y),alpha.T)+np.sum(alpha)
    
    if compute_gap:
        kernel_eval=[np.dot(np.dot(np.dot(alpha,alpha.T),np.dot(y,y.T)),k_i.compute_kernel(X)) for k_i in kernel_list]
        duality_gap=J-np.sum(alpha)+0.5*np.max(kernel_eval)
        
        return J,duality_gap,alpha
    
    return J

def compose_kernels(X,kernel_list,weights):
    """ Compute positive linear combination of kernels 
    """
    return np.sum(np.array([weights[ct]*k_i.compute_kernel(X) for ct,k_i in enumerate(kernel_list)]),axis=0)
    
def compute_gradient(kernel,X,y_outer,alpha):
    """ Compute gradient of MKL objective closed form ; vector
    """
    kernel_mat=kernel.compute_kernel(X)
    
    # utilizes outer product for improved efficiency
    gradient_obj=-0.5*np.dot(np.dot(alpha,np.multiply(y_outer,kernel_mat)),alpha.T)
    
    return gradient_obj


def descent_direction(d_m,mu,gradient_j,grad_mu):
    """ Compute direction of gradient descent ; vector 
    """
    n=len(d_m)
    D=np.zeros(n)
    ongoing_sum=0
    for index in range(0,n):
        if d_m[index]==0 and gradient_j[index]-grad_mu>0:
            D[index]=0
       
    
        elif d_m[index]>0 and index!=mu:
            
            grad_m=-gradient_j[index]+grad_mu
            D[index]=grad_m
            ongoing_sum+=grad_m
            
        
        else:
            D[index]=0
    
    # might not be negative     
    D[mu]=-ongoing_sum

    return D
    
    
def line_search(X,y,kernel_list,D,d_m,gamma_max,disc):
    """ Selects step size to minimize obj value;  
    
        Update from heuristic to exact Armijo's rule 
    """

    if gamma_max==0:
        return gamma_max
    
    # grid of step size begins bigger than 0
    grid=np.arange(0+gamma_max/disc,gamma_max,gamma_max/disc)
    
    min_gamma,min_obj_val=None,10e8
    for gamma_i in grid:
        d_i=d_m+gamma_i*D
        dual_obj_val=compute_dual(X,y,kernel_list,d_m,constant=100,compute_gap=False)
        
        if abs(dual_obj_val)<abs(min_obj_val):
            min_obj_val=dual_obj_val
            min_gamma=gamma_i
        
    
    return min_gamma

def primal_dual_opt(X,y,m,kernel_type,order,gap=10e-4,inner_tol=10e-1,weight_threshold=0.01,maxiter=100, verbose=True):
    """ X feature set, y are class outcomes
        d_m is weight vector  on kernels, alpha is coefficient vector
    """
    
    duality_gap=1
    C=0.01# penalization param
    line_search_steps=25
    n=len(y)
    counter=0
    gamma_max=0
    y_outer=np.outer(y,y)
    
    # optimziation init 
    d_m=np.ones(m)/m
    D=np.ones(m)
    mu=0
    nu=0
    
    if kernel_type=='linear':
        kernel_list=[Kernel(kernel_type=kernel_type)for i in range(1,order+1) ]
    elif kernel_type=='polynomial':
        
        kernel_list=[Kernel(kernel_type,i) for i in range(1,order+1)]
    elif kernel_type=='gaussian':
        kernel_list=[Kernel(kernel_type,bandwidth=i) for i in np.linspace(0.1,1,m)] # gamma hyperparam 

    else:
        print("Not Valid Kernel Type")
        return
    
    # stopping criteria
    while duality_gap>gap and gap>=0:
        old_gap=duality_gap
        if counter>maxiter:
            return d_m
        counter+=1

        # compute svm objective
        J_d,duality_gap,alpha=compute_dual(X,y,kernel_list,d_m,C) 
        if verbose:
            print("Duality",duality_gap)
         
        if abs(duality_gap-old_gap)<gap:
            return d_m
        
        print("Current Guess", d_m)
        # gradient wrt each kernel
        gradient_j=[compute_gradient(i,X,y_outer,alpha) for i in kernel_list] 
        print("Gradient is ",gradient_j)
        grad_mu=gradient_j[mu]
   
        # computes descent direction ; normalized for equality constraints 
        D=descent_direction(d_m,mu,gradient_j,grad_mu)
        
        norm_D=np.sqrt(D.dot(D))
        D=D/norm_D
        print("Descent Direction is ",D)
        mu=np.argmax(d_m)

        print(d_m)

        
        J_hat=0
        d_hat=d_m
        D_hat=D
        
        
        # descent direction update
        inner_iter=0
        while J_hat+inner_tol<J_d or inner_iter<maxiter:
            inner_iter+=1 
            
            # indices where descent direction is negative 
            nonzero_D=np.where(D_hat<0)[0]
            
            if len(nonzero_D)==0:
                # if none we have reached local minimia 
                gamma_max=0  
                
            else:
                # else step gamma size
                gamma_max=np.min(-d_hat[nonzero_D]/D_hat[nonzero_D])
      
            D=D_hat
            d_m=d_hat

            d_hat=d_m+gamma_max*D
            d_hat[d_hat<weight_threshold]=0
                
            # need to fix descent step here 
            #######################
            D_hat[mu]=D[mu]-D[nu]
            D_hat[nu]=0
            
            J_hat=compute_dual(X,y,kernel_list,d_hat,C,compute_gap=False)
            print("J_hat",J_hat,"J_d",J_d)
            #####################
            
        # line search in descent direction  
        gamma_step=line_search(X,y,kernel_list,D,d_m,gamma_max,disc=line_search_steps)
        
        d_m=(d_m+gamma_step*D)
        
        # normalize and drop threshold
        d_m[d_m<weight_threshold]=0
        d_m=d_m/np.sum(d_m)
        
        print("NEW",d_m)
       
        
    return d_m
    

In [104]:
def test_mkl(verbose=True):
    
    n=100 # data set size
    
    m=5 #6 num kernels
    
    y=np.sign(np.random.uniform(-1,1,size=n)) # sample class labels 1,-1
    x=np.random.rand(n,10) # # sample features
    kernel_type='polynomial'
    
  
    d_m=np.round(primal_dual_opt(x,y,m,kernel_type,order=m,verbose=verbose),2)
    
    
    return d_m 

test_mkl()

Duality 1189679.2650627315
Current Guess [0.2 0.2 0.2 0.2 0.2]
Gradient is  [-18.738459840682864, -80.49719973743147, -359.5692954701435, -1676.6685636008094, -8165.2137079557215]
Descent Direction is  [-0.77509096  0.00468979  0.02588175  0.12589863  0.61862079]
[0.2 0.2 0.2 0.2 0.2]
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 6533.972298681651 J_d 2221.8232971641037
J_hat 653

array([0.  , 0.2 , 0.21, 0.23, 0.36])

In [89]:
a=np.array([])
len(np.where(a<0)[0])

0

In [ ]:
a=np.array([1,2,3])
b=np.array([4,5,6])

np.outer(a,a)

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

### Issues at the moment
- Gradient isn't always negative; which means there is an issue in computation as this corresponds to a decrease

SVM includes NAN values
- Gaussian Kernels give equidistant results 